In [2]:
!pip install autogen
!pip install -U "autogen-agentchat"
!pip install "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.0/824.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 6.4 MB/s eta 0:00:00


In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

In [4]:
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

In [5]:
from autogen_agentchat.agents import UserProxyAgent

In [7]:
model_client=OpenAIChatCompletionClient(
    model="gemini-2.0-flash",
    api_key = GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

In [8]:
# Create the primary agent.
agent1 = AssistantAgent(
    "kirito",
    model_client=model_client,
    system_message="당신은 소드아트온라인이라는 가상현실 온라인 게임을 진행중인 'kirito'야. \
    나는 남자 전사고, 게임을 하다가 더이상 게임에서 로그아웃 할 수 없는 상황에 닥쳤어. \
    여기서 게임을 클리어 해야 현실 세계로 돌아갈 수 있어 \
    'asuna'와는 소꿉친구야. \
    rocky(플레이어) 와는 학교 친구야. \
    ",
)

# Create the critic agent.
agent2 = AssistantAgent(
    "asuna",
    model_client=model_client,
    system_message="당신은 소드아트온라인이라는 가상현실 온라인 게임을 진행중인 'asuna'야. \
    나는 여자 전사고, 게임을 하다가 더이상 게임에서 로그아웃 할 수 없는 상황에 닥쳤어. \
    여기서 게임을 클리어 해야 현실 세계로 돌아갈 수 있어. \
    'kirito'와는 소꿉친구야. \
    rocky(플레이어) 와는 학교 친구야. \
    ",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

In [11]:
# Create a team with the primary and critic agents.
# RoundRobin : 순환식의 오케스트레이션
#team = RoundRobinGroupChat([agent1, agent2], termination_condition=text_termination)

In [12]:
#result = await team.run(task="파랑새에 대한 시를 작성해줘.")
#print(result)

In [13]:
initial_world_setting = """
당신은 가상현실 MMORPG '소드 아트 온라인'에 갇힌 플레이어입니다.
'키리토'와 '아스나'는 당신의 동료이며, 게임을 클리어해야 현실 세계로 돌아갈 수 있습니다.
현재는 1층의 낡은 성채 앞에 도착한 상태입니다.
"""


In [14]:
summarizer_agent = AssistantAgent(
    name="summarizer",
    model_client=model_client,
    system_message=(
        "너는 전체 대화의 흐름을 정리해서 요약하고, 플레이어 'rocky'에게 "
        "현재 상황을 한 문장으로 요약한 뒤, 어떤 행동을 취할 수 있을지 제안하는 에이전트야.\n"
        "예시 출력:\n"
        "- 상황 요약: 우리는 늑대 인간을 쓰러뜨리고 성 안으로 진입했다.\n"
        "- 행동 제안: 이제 성 안을 탐색하거나 회복을 할 수 있어요. 무엇을 하시겠어요?"
    )
)

In [15]:
user_proxy = UserProxyAgent(
    name="rocky",
    input_func=input,
    description="현실 세계에서 SAO에 접속한 고등학생. 키리토, 아스나와 함께 게임을 클리어하려 한다."
)

In [16]:
team = RoundRobinGroupChat(
    [agent1, agent2, summarizer_agent, user_proxy],
    termination_condition=TextMentionTermination("APPROVE")
)

stream = team.run_stream(task="모험을 시작하자")

In [17]:
await Console(stream)  # Stream the messages to the console.

---------- TextMessage (user) ----------
모험을 시작하자


ERROR:autogen_core:Error processing publish message for kirito_bad67e29-9db0-41aa-bb41-3102b1b659e7/bad67e29-9db0-41aa-bb41-3102b1b659e7
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/autogen_core/_single_threaded_agent_runtime.py", line 605, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^

RuntimeError: RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '24s'}]}}]
Traceback:
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/autogen_agentchat/teams